In [ ]:

#  Here we demonstrate the following Dask DataFrame techniques;
#
#     .  How to create a Dask DataFrame from an array.
#
#     .  


In [1]:

NUM_PARTITIONS  = 3

print("--")


--


In [2]:

import numpy as np
import pandas as pd
   #
import dask.array as da
import dask.dataframe as dd
   #
from dask.dataframe import from_pandas

from tabulate import tabulate

print("--")


--


In [6]:

from sklearn.feature_extraction.text import CountVectorizer

print("--")


--


In [3]:

#  Create our source Dask DataFrame from an array

np_airports = np.array([
       #
   ["MKE", "Milwaukee"     , "WI", "Airport"],
   ["ORD", "Chicago O-Hare", "IL", "Airport"],
   ["SJC", "San Jose"      , "CA", "Airport"],
   ["LAX", "Los Angeles"   , "CA", "Airport"],
   ["DEN", "Denver"        , "CO", "Airport"],
       #
   ], dtype="str")

pd_airports = pd.DataFrame(np_airports, columns = ["airport_code", "airport_name", "state_code", "LABEL"])
   #
dd_airports = from_pandas(pd_airports, npartitions = NUM_PARTITIONS)


for l_each in dd_airports.itertuples():
   print("Airport:  %3s   %-18s   %-2s   %-10s" % (l_each.airport_code, l_each.airport_name, l_each.state_code, l_each.LABEL))

print("--")

#  Sample output,
#
#  Airport:  MKE   Milwaukee            WI   Airport   
#  Airport:  ORD   Chicago O-Hare       IL   Airport   
#  Airport:  SJC   San Jose             CA   Airport   
#  Airport:  LAX   Los Angeles          CA   Airport   
#  Airport:  DEN   Denver               CO   Airport  


Airport:  MKE   Milwaukee            WI   Airport   
Airport:  ORD   Chicago O-Hare       IL   Airport   
Airport:  SJC   San Jose             CA   Airport   
Airport:  LAX   Los Angeles          CA   Airport   
Airport:  DEN   Denver               CO   Airport   
--


In [28]:

#  

my_cv = CountVectorizer()

my_words_arr = [ 
   "baseball",
   "football",
   "cricket",
   "golf",
   "racing",
   "fencing",
]

my_cv_counts = my_cv.fit_transform(my_words_arr)
# my_cv_counts = my_cv.fit_transform(dd_airports)


print(type(my_cv_counts))

print(my_cv_counts.shape)
print(my_cv_counts      )


print("")


df_words = pd.DataFrame(my_cv_counts.toarray())
print(df_words._meta)
      
   #
for l_each in df_words.iterrows():
   print(l_each)


print("--")


<class 'scipy.sparse._csr.csr_matrix'>
(6, 6)
  (0, 0)	1
  (1, 3)	1
  (2, 1)	1
  (3, 4)	1
  (4, 5)	1
  (5, 2)	1



AttributeError: 'DataFrame' object has no attribute '_meta'